In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
data=pd.read_csv("/content/drive/MyDrive/final_year_project/final_dataset_bse.csv")

<ipython-input-4-ffd83b337010>:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("/content/drive/MyDrive/final_year_project/final_dataset_bse.csv")


In [5]:
data_e=pd.read_csv("/content/drive/MyDrive/final_year_project/Equity (2).csv")

In [6]:
data.columns

Index(['Date', 'Open Price', 'High Price', 'Low Price', 'Close Price', 'WAP',
       'No.of Shares', 'No. of Trades', 'Total Turnover (Rs.)',
       'Deliverable Quantity', '% Deli. Qty to Traded Qty', 'Spread High-Low',
       'Spread Close-Open', 'Co Name'],
      dtype='object')

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3613305 entries, 0 to 3613304
Data columns (total 14 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   Date                       object 
 1   Open Price                 float64
 2   High Price                 float64
 3   Low Price                  float64
 4   Close Price                float64
 5   WAP                        float64
 6   No.of Shares               int64  
 7   No. of Trades              int64  
 8   Total Turnover (Rs.)       float64
 9   Deliverable Quantity       float64
 10  % Deli. Qty to Traded Qty  float64
 11  Spread High-Low            float64
 12  Spread Close-Open          float64
 13  Co Name                    object 
dtypes: float64(10), int64(2), object(2)
memory usage: 385.9+ MB


In [8]:
data=data.dropna()

In [9]:
#Data Cleaning- Removing companies with less than 2 months data
companies_to_remove = data['Co Name'].value_counts()[data['Co Name'].value_counts() < 50].index.tolist()
data = data[~data['Co Name'].isin(companies_to_remove)]

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3598017 entries, 0 to 3613304
Data columns (total 14 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   Date                       object 
 1   Open Price                 float64
 2   High Price                 float64
 3   Low Price                  float64
 4   Close Price                float64
 5   WAP                        float64
 6   No.of Shares               int64  
 7   No. of Trades              int64  
 8   Total Turnover (Rs.)       float64
 9   Deliverable Quantity       float64
 10  % Deli. Qty to Traded Qty  float64
 11  Spread High-Low            float64
 12  Spread Close-Open          float64
 13  Co Name                    object 
dtypes: float64(10), int64(2), object(2)
memory usage: 411.8+ MB


In [11]:
data_e.columns

Index(['Security Code', 'Issuer Name', 'Security Id', 'Security Name',
       'Status', 'Group', 'Face Value', 'ISIN No', 'Industry', 'Instrument',
       'Sector Name', 'Industry New Name', 'Igroup Name', 'ISubgroup Name'],
      dtype='object')

In [12]:
data_e=data_e.dropna()

In [13]:
data_e.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4244 entries, 0 to 4243
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Security Code      4244 non-null   int64  
 1   Issuer Name        4244 non-null   object 
 2   Security Id        4244 non-null   object 
 3   Security Name      4244 non-null   object 
 4   Status             4244 non-null   object 
 5   Group              4244 non-null   object 
 6   Face Value         4244 non-null   float64
 7   ISIN No            4244 non-null   object 
 8   Industry           4244 non-null   object 
 9   Instrument         4244 non-null   object 
 10  Sector Name        4244 non-null   object 
 11  Industry New Name  4244 non-null   object 
 12  Igroup Name        4244 non-null   object 
 13  ISubgroup Name     4244 non-null   object 
dtypes: float64(1), int64(1), object(12)
memory usage: 464.3+ KB


In [14]:
merged_df = pd.merge(data, data_e, left_on='Co Name', right_on='Security Code', how='left')
# Add a new column 'Issuer Name' to df1 with values from df2
data['Issuer Name'] = merged_df['Issuer Name']
data['Sector']=merged_df['Sector Name']

In [15]:
print(data.columns)

Index(['Date', 'Open Price', 'High Price', 'Low Price', 'Close Price', 'WAP',
       'No.of Shares', 'No. of Trades', 'Total Turnover (Rs.)',
       'Deliverable Quantity', '% Deli. Qty to Traded Qty', 'Spread High-Low',
       'Spread Close-Open', 'Co Name', 'Issuer Name', 'Sector'],
      dtype='object')


In [16]:
# from itertools import combinations
# from tqdm import tqdm
# unique_companies = data["Co Name"].unique()
# correlation_dict = {}

# # Iterate through pairs of unique companies and compute the cross-correlation
# with tqdm(total=len(list(combinations(unique_companies, 2))) // 2) as pbar:
#     for company1, company2 in combinations(unique_companies, 2):
#         if (company1, company2) not in correlation_dict:
#             condition1 = (data['Co Name'] == company1)
#             condition2 = (data['Co Name'] == company2)

#             prices_close = data.loc[condition1, 'Close Price'].tolist()
#             prices_wap = data.loc[condition1, 'WAP'].tolist()
#             prices2_close = data.loc[condition2, 'Close Price'].tolist()
#             prices2_wap = data.loc[condition2, 'WAP'].tolist()

#             # Calculate cross-correlation with a lag of 1 week (7 days)
#             cross_correlation_close = np.correlate(prices_close, prices2_close, mode='full')
#             cross_correlation_wap = np.correlate(prices_wap, prices2_wap, mode='full')

#             # Normalize the cross-correlation values
#             max_correlation_close = cross_correlation_close.max()
#             max_correlation_wap = cross_correlation_wap.max()

#             # Average of Close Price and WAP correlations
#             avg_correlation = (max_correlation_close + max_correlation_wap) / 2

#             correlation_dict[(company1, company2)] = avg_correlation
#             pbar.update(1)

# # Create the correlation matrix
# correlation_matrix2 = np.zeros((len(unique_companies), len(unique_companies)))

# for i, company1 in enumerate(unique_companies):
#     for j, company2 in enumerate(unique_companies):
#         if i < j:
#             correlation_matrix2[i, j] = correlation_dict[(company1, company2)]
#         elif i > j:
#             correlation_matrix2[i, j] = correlation_dict[(company2, company1)]

# # Fill the diagonal elements with 1 since the cross-correlation with itself is 1
# np.fill_diagonal(correlation_matrix2, 1)

# # Print the final correlation matrix
# print("Average Cross-Correlation Matrix:")
# print(pd.DataFrame(correlation_matrix2, columns=unique_companies, index=unique_companies))

In [17]:
correlation_tables = []
pivot_table = data.pivot_table(index='Date', columns='Issuer Name', values='Close Price')
correlation_tables.append(pivot_table)
correlation_matrix = pivot_table.corr()
print(correlation_matrix)

Issuer Name                    Aurum PropTech Limited   Gland Pharma Limited  \
Issuer Name                                                                    
 Aurum PropTech Limited                      1.000000               0.229501   
 Gland Pharma Limited                        0.229501               1.000000   
 Max India Limited                          -0.390070              -0.102399   
20 Microns Limited                           0.444748               0.564358   
360 ONE WAM LIMITED                          0.140618              -0.010994   
...                                               ...                    ...   
eClerx Services Limited                      0.573920              -0.388500   
eMudhra Limited                              0.736370               0.555782   
i Power Solutions India Ltd.                 0.557457                    NaN   
iStreet Network Limited                      0.727969               0.262887   
ramkrishna forgings Ltd.                

In [ ]:
start = 0.5
end = 1.0
step = 0.01

threshold = []
degree = []

for t in np.arange(start, end + step, step):
    threshold.append(t)

    # Create an empty list to store the edges
    edges = []

    # Iterate over the correlation matrix and extract the edges based on the threshold
    for i in range(len(correlation_matrix.columns)):
        for j in range(i + 1, len(correlation_matrix.columns)):
            weight = correlation_matrix.iloc[i, j]  # Access the correlation using .iloc
            if abs(weight) >= t:
                edges.append([i, j, weight])

    x = len(edges)
    degree.append(x)

# Plot the data
plt.plot(threshold, degree, label='Threshold Vs Edges', color='blue')

# Add labels for the x and y axes
plt.xlabel('Threshold')
plt.ylabel('Edges')

# Add a legend
plt.legend()
plt.show()

In [ ]:
# Set the threshold value
threshold = 0.9

  # Get the company names from the column names of the correlation matrix
company_names = correlation_matrix.columns.tolist()

  # Create an empty list to store the edges
edges = []

  # Iterate over the correlation matrix and extract the edges based on the threshold
for i in range(len(company_names)):
  for j in range(i + 1, len(company_names)):
    from_edge = company_names[i]
    to_edge = company_names[j]
    weight = correlation_matrix.iloc[i, j]
    if abs(weight) >= threshold:
      edges.append([from_edge, to_edge, weight])
# Create a DataFrame from the edges list
edge_list = pd.DataFrame(edges, columns=['Source', 'Target', 'Weight'])

# Export the edge list as a CSV file
edge_list.to_csv('bseedge_list500.csv', index=False)

In [ ]:
print(edge_list)

In [ ]:
start = 0.5
end = 1.0
step = 0.01

threshold = []
modularity = []

for t in np.arange(start, end + step, step):
    threshold.append(t)
    company_names = correlation_matrix.columns.tolist()

    # Create an empty list to store the edges
    edges = []

      # Iterate over the correlation matrix and extract the edges based on the threshold
    for i in range(len(company_names)):
      for j in range(i + 1, len(company_names)):
        from_edge = company_names[i]
        to_edge = company_names[j]
        weight = correlation_matrix.iloc[i, j]
        if abs(weight) >= t:
          edges.append([from_edge, to_edge, weight])
    # Create a DataFrame from the edges list
    edge_list = pd.DataFrame(edges, columns=['Source', 'Target', 'Weight'])
    # Create an empty graph
    G = nx.Graph()
    # Iterate over the edge list and add nodes and weighted edges to the graph
    for row in edge_list.itertuples():
        source = row.Source
        target = row.Target
        weight = row.Weight
        G.add_node(source)
        G.add_node(target)
        G.add_edge(source, target, weight=weight)
    # Modularity
    communities = list(nx.community.greedy_modularity_communities(G))
    modularity.append(nx.community.modularity(G, communities))

# Plot the data
plt.plot(threshold, modularity, label='Threshold Vs Modularity', color='blue')

# Add labels for the x and y axes
plt.xlabel('Threshold')
plt.ylabel('Modularity')

# Add a legend
plt.legend()
plt.show()

In [ ]:
max_modularity_index = np.argmax(modularity)
max_modularity = modularity[max_modularity_index]
corresponding_threshold = threshold[max_modularity_index]

print(f"Maximum Modularity: {max_modularity} at Threshold: {corresponding_threshold}")

In [ ]:
import networkx as nx

# Create an empty graph
G = nx.Graph()

# Iterate over the edge list and add nodes and weighted edges to the graph
for row in edge_list.itertuples():
    source = row.Source
    target = row.Target
    weight = row.Weight
    G.add_node(source)
    G.add_node(target)
    G.add_edge(source, target, weight=weight)

# Modularity
communities = list(nx.community.greedy_modularity_communities(G))
modularity = nx.community.modularity(G, communities)
print(modularity)


In [ ]:
# Calculate network science parameters
# Degree Centrality
degree_centrality = nx.degree_centrality(G)
max_degree_node = max(degree_centrality, key=degree_centrality.get)
print("Company with Maximum Degree Centrality:")
print("Company Name:", max_degree_node)
print("Degree Centrality:", degree_centrality[max_degree_node])
# Betweenness Centrality
betweenness_centrality = nx.betweenness_centrality(G)
max_betweenness_node = max(betweenness_centrality, key=betweenness_centrality.get)
print("Company with Maximum Betweenness Centrality:")
print("Company Name:", max_betweenness_node)
print("Betweenness Centrality:", betweenness_centrality[max_betweenness_node])

# Closeness Centrality
closeness_centrality = nx.closeness_centrality(G)
max_closeness_node = max(closeness_centrality, key=closeness_centrality.get)
print("Company with Maximum Closeness Centrality:")
print("Company Name:", max_closeness_node)
print("Closeness Centrality:", closeness_centrality[max_closeness_node])

# Eigenvector Centrality
eigenvector_centrality = nx.eigenvector_centrality(G)
max_eigenvector_node = max(eigenvector_centrality, key=eigenvector_centrality.get)
print("Company with Maximum Eigenvector Centrality:")
print("Company Name:", max_eigenvector_node)
print("Eigenvector Centrality:", eigenvector_centrality[max_eigenvector_node])

# Clustering Coefficient
clustering_coefficient = nx.clustering(G)
max_clustering_node = max(clustering_coefficient, key=clustering_coefficient.get)
print("Company with Maximum Clustering Coefficient:")
print("Company Name:", max_clustering_node)
print("Clustering Coefficient:", clustering_coefficient[max_clustering_node])


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Create or load your graph (G) using NetworkX
# ...

# Perform community detection using the Louvain method
communities = list(nx.community.greedy_modularity_communities(G))
modularity = nx.community.modularity(G, communities)

# Create a mapping of nodes to communities
community_mapping = {}
for i, community in enumerate(communities):
    for node in community:
        community_mapping[node] = i

# Extract positions of nodes in the graph
pos = nx.spring_layout(G)

# Create a custom color map for communities
num_communities = len(communities)
cmap = plt.get_cmap('jet', num_communities)

# Draw nodes and edges
nx.draw(G, pos, with_labels=True, node_size=50)
nx.draw_networkx_edges(G, pos, alpha=0.5)

# Draw nodes with colors based on communities using the custom color map
node_colors = [community_mapping[node] for node in G.nodes()]
node_color_values = [cmap(i) for i in node_colors]
nx.draw_networkx_nodes(G, pos, node_color=node_color_values, node_size=50)

# Add a legend
legend_labels = {community_id: f"Community {community_id}" for community_id in set(community_mapping.values())}

# Print the modularity and the number of communities
print("Modularity:", modularity)
print("Number of Communities:", num_communities)
nx.write_graphml(G, "graph.gephi.graphml")

In [ ]:
for i, community in enumerate(communities):
    print(f"Community {i + 1}: {list(community)}")

In [ ]:
import matplotlib.pyplot as plt

# Assuming you already have communities obtained using greedy_modularity_communities
communities = list(nx.community.greedy_modularity_communities(G))

# Function to find unique sectors within a community
def find_unique_sectors(community):
    unique_sectors = set()
    for comm in list(community):
        sectors = ', '.join(data.loc[data['Issuer Name'] == comm, 'Sector'].unique())
        unique_sectors.add(sectors)
    return unique_sectors

# Plot bar graphs for sector distribution within each community
for i, community in enumerate(communities):
    unique_sectors = find_unique_sectors(community)
    print(f"Community {i + 1} ({len(community)} nodes): {list(community)}")
    print(f"Unique Sectors: {unique_sectors}")

    # Count occurrences of each sector within the community
    sector_counts = {}
    for comm in list(community):
            sectors = ', '.join(data.loc[data['Issuer Name'] == comm, 'Sector'].unique())
            #print(sectors)
            if sectors not in sector_counts:
                sector_counts[sectors] = 1
            else:
                sector_counts[sectors] += 1
            unique_sectors.add(sectors)

    # Plot the bar graph
    plt.bar(sector_counts.keys(), sector_counts.values())
    plt.xlabel('Sector')
    plt.ylabel('Count')
    plt.title(f'Sector Distribution in Community {i + 1}')
    plt.xticks(rotation=45)
    plt.show()

    print()


In [ ]:
# Assuming you already have communities obtained using greedy_modularity_communities
communities = list(nx.community.greedy_modularity_communities(G))

# Function to find sub-communities within a given community
def find_sub_communities(community, G):
    subgraph = G.subgraph(community)
    sub_communities = list(nx.community.greedy_modularity_communities(subgraph))
    return sub_communities

# Find sub-communities for each community
all_sub_communities = []
for i, community in enumerate(communities):
    sub_communities = find_sub_communities(community, G)
    all_sub_communities.append(sub_communities)

    print(f"Sub-Communities for Community {i + 1} (Total {len(community)} nodes):")
    for j, sub_community in enumerate(sub_communities):
        print(f"Sub-Community {j + 1} ({len(sub_community)} nodes): {list(sub_community)}")

In [ ]:
# Function to find sub-communities within a given community
def find_sub_communities(community, G):
    subgraph = G.subgraph(community)
    sub_communities = list(nx.community.greedy_modularity_communities(subgraph))
    return sub_communities

# Find sub-communities for each community
all_sub_communities = []
for i, community in enumerate(communities):
    sub_communities = find_sub_communities(community, G)
    all_sub_communities.append(sub_communities)

    print(f"Sub-Communities for Community {i + 1} (Total {len(community)} nodes):")
    for j, sub_community in enumerate(sub_communities):
        unique_sectors = set()
        sec = {}
        print(f"Sub-Community {j + 1} ({len(sub_community)} nodes): {list(sub_community)}")
        for comm in list(sub_community):
            sectors = ', '.join(data.loc[data['Issuer Name'] == comm, 'Sector'].unique())
            #print(sectors)
            if sectors not in sec:
                sec[sectors] = 1
            else:
                sec[sectors] += 1
            unique_sectors.add(sectors)
        print("Unique Sectors:", unique_sectors)
        print("Sectors:", sec)


In [ ]:
import matplotlib.pyplot as plt

# Assuming you already have communities obtained using greedy_modularity_communities
communities = list(nx.community.greedy_modularity_communities(G))

# Function to find unique sectors within a community
def find_unique_sectors(community):
    unique_sectors = set()
    for comm in list(community):
        sectors = ', '.join(data.loc[data['Issuer Name'] == comm, 'Sector'].unique())
        unique_sectors.add(sectors)
    return unique_sectors

# Plot bar graphs for sector distribution within each sub-community
for i, community in enumerate(communities):
    print(f"Community {i + 1} ({len(community)} nodes): {list(community)}")

    # Find sub-communities within the current community
    sub_communities = list(nx.community.greedy_modularity_communities(G.subgraph(community)))

    # Plot sector distribution for each sub-community
    for j, sub_community in enumerate(sub_communities):
        unique_sectors = find_unique_sectors(sub_community)
        print(f"Sub-Community {j + 1} ({len(sub_community)} nodes): {list(sub_community)}")
        print(f"Unique Sectors: {unique_sectors}")

        # Count occurrences of each sector within the sub-community
        sector_counts = {}
        for comm in list(sub_community):
            sectors = ', '.join(data.loc[data['Issuer Name'] == comm, 'Sector'].unique())
            if sectors not in sector_counts:
                sector_counts[sectors] = 1
            else:
                sector_counts[sectors] += 1

        # Plot the bar graph for the sub-community
        plt.bar(sector_counts.keys(), sector_counts.values())
        plt.xlabel('Sector')
        plt.ylabel('Count')
        plt.title(f'Sector Distribution in Sub-Community {j + 1} (Community {i + 1})')
        plt.xticks(rotation=45)
        plt.show()

    print()


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt


# List of community detection algorithms to test communities = list(nx.community.greedy_modularity_communities(G))
algorithms = [
    ("Girvan-Newman", nx.community.girvan_newman(G)),
    ("Louvain", nx.community.greedy_modularity_communities(G)),
    ("Label Propagation", nx.community.label_propagation_communities(G)),
    ("K-Clique", nx.community.k_clique_communities(G, 10)),  # Adjust k as needed
    ("Modularity", nx.community.greedy_modularity_communities(G)),  # Modularity-based detection
    # Add more algorithms as needed
]

# Dictionary to store the number of communities detected for each algorithm
community_counts = {}
# Iterate through the algorithms and print the number of communities
for algorithm_name, algorithm in algorithms:
    if isinstance(algorithm, list):
        communities = algorithm
    else:
        communities = list(algorithm)

    num_communities = len(communities)
    community_counts[algorithm_name] = num_communities
    print(f"{algorithm_name} detected {num_communities} communities")


# Plot the results
plt.figure(figsize=(12, 6))
plt.bar(community_counts.keys(), community_counts.values())
plt.xlabel("Community Detection Algorithm")
plt.ylabel("Number of Communities Detected")
plt.title("Number of Communities Detected by Different Algorithms")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()